Distiled Roberta tokenizer

In [26]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")
tokenizer("Hello world")['input_ids']
tokenizer(" Hello world")['input_ids']  

[0, 20920, 232, 2]

In [37]:
import torch
from transformers import RobertaTokenizer, RobertaForMaSkedLM

model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

sequence = f"What the {tokenizer.mask_token} is going on?" # "The world will end in <mask>"

input_seq = tokenizer.encode(sequence, return_tensors='pt') # tensor([[0, 133, 232, 40, 253, 11, 50264, 2]])
mask_token_index = torch.where(input_seq == tokenizer.mask_token_id)[1] # (tensor([0]), tensor([6])) - we only want the the 2nd dimension
token_logits = model(input_seq).logits
masked_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(masked_token_logits, 5, dim=1).indices[0].tolist()

# print('sequence:', sequence)
# print('input_seq:', input_seq)
# print('mask_token_index:', mask_token_index)
# print('token_logits:', token_logits)
# print('masked_token_logits:', masked_token_logits)
print('top_5_tokens:', top_5_tokens)


top_5_tokens: [7105, 17835, 26536, 11141, 39072]


In [38]:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

What the  hell is going on?
What the  heck is going on?
What the  fuck is going on?
What the  Hell is going on?
What the  HELL is going on?


In [53]:
from itertools import permutations
Sk = [1,2,4,5,6]
Wk = ['A','B','C','D']
if len(Wk)<len(Sk):
    perms = list(permutations(Sk,len(Wk)))
else:
    perms = list(permutations(Wk,len(Sk)))

In [ ]:
for 

In [13]:
from itertools import permutations

def candidate_set_generation(xi, pi0, Ti, T0i, F):
    # Initialize candidate sentence with masked tokens
    c0 = '[MASK]' * len(pi0)
    Ci = {c0}
    # Find shared POS tags between pi and pi0
    Tshared = Ti.intersection(T0i)
    for tk in Tshared:
        # Find words in xi that are tagged with tk
        Wk = [word for word, pos in xi if pos == tk]
        # Find positions in pi0 where tk occurs
        Sk = [i for i, pos in enumerate(pi0) if pos == tk]
        # Generate all possible assignments of words in Wk to positions in Sk
        Ak = list(permutations(Wk, len(Sk)))
        if not Ak:
            continue
        # Generate new candidate sentences by filling in masked tokens with word assignments
        new_Ci = set()
        for cj in Ci:
            for a in Ak:
                c0j = [c for c in cj]
                for i, word in zip(Sk, a):
                    c0j[i] = word
                new_Ci.add(' '.join(c0j))
        Ci = new_Ci
    # Apply mask-filling model to each candidate sentence and return the set of filled sentences
    return {F(cj) for cj in Ci}


In [23]:
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

# Load pre-trained RoBERTa model and tokenizer
model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# Set model to evaluation mode
model.eval()

# Define mask-filling function using RoBERTa
def fill_mask(text):
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors='pt')
    # Find the [MASK] token index
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    # Get model predictions for the masked tokens
    with torch.no_grad():
        output = model(input_ids)
        print(output)
        mask_token_logits = output.logits[0, mask_token_index, :]
    # Get the top predicted tokens and convert them to strings
    top_tokens = torch.topk(mask_token_logits, k=5, dim=1).indices[0].tolist()
    top_token_strings = tokenizer.convert_ids_to_tokens(top_tokens)
    # Replace the [MASK] token with each of the top predicted tokens and return the resulting texts
    # print(top_token_strings)
    return [text[:mask_token_index[0].item()] + token + text[mask_token_index[0].item()+1:] for token in top_token_strings]

In [24]:
# Example inputs
xi = [('The', 'DT'), ('fuck', '[MASK]'), ('brown', 'JJ'), ('fox', 'NN')]
pi = ['The', 'quick', 'brown', 'fox']
pi0 = ['DT', 'JJ', 'JJ', 'NN']
Vr = {'brown': 1}
Ti = {'DT', 'JJ', 'NN'}
T0i = {'DT', 'JJ', 'NN'}

# Call candidate_set_generation function
candidate_sentences = candidate_set_generation(xi, pi, pi0, Vr, Ti, T0i, fill_mask)

# Print the set of candidate sentences
print(candidate_sentences)


MaskedLMOutput(loss=None, logits=tensor([[[ 3.4195e+01, -4.1623e+00,  2.0543e+01,  ...,  4.6145e-01,
           3.5389e+00,  1.2653e+01],
         [ 1.3564e+00, -5.3182e+00,  1.0158e+01,  ..., -6.0494e+00,
          -4.8035e+00,  2.2738e+00],
         [-2.5319e+00, -6.0481e+00,  8.3431e+00,  ..., -7.0483e+00,
          -5.6234e+00,  9.2884e-02],
         ...,
         [ 3.4954e-01, -4.4015e+00,  1.4322e+01,  ..., -3.1012e+00,
          -2.6098e+00, -3.2432e-02],
         [-1.0425e+00, -4.7949e+00,  1.4543e+01,  ..., -3.0679e+00,
          -1.2790e+00, -2.6544e+00],
         [ 1.5625e+01, -5.6885e+00,  2.0755e+01,  ..., -3.3742e+00,
          -8.6062e-01,  4.5708e+00]]]), hidden_states=None, attentions=None)


IndexError: index 0 is out of bounds for dimension 0 with size 0

References

https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta